In [1]:
import time
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=6

from numpy import empty
from numpy import load
#import tensorflow as tf

import wandb
from src.training_setup import kfold_cv, train_holdout

# Set CPU as available physical device
import itertools
import numpy as np
import pandas as pd
import itertools
import scipy.fftpack

Set random seed

In [2]:
import sys
sys.path.insert(1, '.')
my_seed = 19951008
import random
random.seed(my_seed)
from numpy.random import seed
seed(my_seed)
from tensorflow import random
random.set_seed(my_seed)

Load data:

In [3]:
%%time
data = np.load('/media/hdd1/khaled/npz_files/final_version/numpy_train_obj_unbalanced.npz', allow_pickle=True)

vectors = []
for x in list(data.keys()):
    vectors.append(data[x])
X, y, X_spk_labels, X_spk_labels_aug, X_aug, y_aug = vectors
X_spk_labels_aug.shape
data.close()
data = None
vectors = None
del data, vectors

X_mel = empty(X.shape, dtype='object')
for i in range(X.shape[0]):
    X_mel[i] = scipy.fftpack.idct(X[i])
X_aug_mel = empty(X_aug.shape, dtype='object')
for i in range(X_aug.shape[0]):
    X_aug_mel[i] = scipy.fftpack.idct(X_aug[i])

CPU times: user 42.8 s, sys: 21.5 s, total: 1min 4s
Wall time: 1min 4s


Load pandas dataframes

In [4]:
train_metadata = pd.read_csv('age-train.txt')
test_metadata = pd.read_csv('age-test.txt')
title_only_metadata = pd.read_csv('age-title_only.txt')

Load test set:

In [5]:
%%time
data = np.load('/media/hdd1/khaled/npz_files/final_version/test_data.npz', allow_pickle=True)

vectors = []
for x in list(data.keys()):
    vectors.append(data[x])
X_test, y_test, X_spk_labels_test = vectors
data.close()
data = None
vectors = None
del data, vectors


X_test_mel = empty(X.shape, dtype='object')
for i in range(X_test.shape[0]):
    X_test_mel[i] = scipy.fftpack.idct(X_test[i])

CPU times: user 6.6 s, sys: 2.34 s, total: 8.94 s
Wall time: 8.95 s


In [11]:
def get_correct_recordings_index(spk_labels):
    print('get_correct_recordings_index >>>')
    spk_labels_dict = {i:spk_labels.count(i) for i in set(spk_labels)}
    least_freq_spk = min(list(spk_labels_dict.values()))
    print(least_freq_spk)
    speaker_indexes = []
    frequency_spk_labels_dict = {}
    for x in set(spk_labels):
        frequency_spk_labels_dict[x] = 0
    for index, spk_id in enumerate(spk_labels):
        frequency_spk_labels_dict[spk_id] += 1
        if frequency_spk_labels_dict[spk_id] > least_freq_spk:
            next
        else:
            speaker_indexes.append(index)
    print('get_correct_recordings_index <<<')
    return speaker_indexes

In the test set, currently, there are all labeled pairs person-yt videos, however we need to balance them so that each speaker has the same weight. The first step is to identify the ids of interest:

In [7]:
%%time
X_spk_video_labels_test=X_spk_labels_test
X_spk_labels_test = [''.join(x.split('-')[1:]) for x in X_spk_video_labels_test]
test_ids_balanced = get_correct_recordings_index(X_spk_labels_test)
len(test_ids_balanced)

get_correct_recordings_index >>>
1
get_correct_recordings_index <<<
CPU times: user 351 ms, sys: 7.69 ms, total: 359 ms
Wall time: 354 ms


958

This means that we'll have 958 test records!

In [9]:
X_test = X_test[test_ids_balanced]
y_test = y_test[test_ids_balanced]
X_test_mel = X_test_mel[test_ids_balanced]

In [10]:
X_test.shape, y_test.shape, X_test_mel.shape

((958,), (958,), (958,))

# Model train
## CNN 1D : Multi input - Multi output

In [11]:
# Params
norm_strat_to_evaluate = ['sub_mean_dataloader']
y_strategy = ['']
l_reg = [0.0]
filter_n = [30]
kernel_size = [3]
pool_size = [(2)]
dense_n = [256]
batch_size = [128]
lr = [0.01]
optimizer = ['adam']
second_dense_n = [128]
data_augmentation = [True]
selective_data_aug = [False]
loss = ['mse_plus_cross']
block_list = [[1, 1, 1]]
global_avg = [True]
train_combinations = list(itertools.product(['cnn_resnet_1d'],
                                            norm_strat_to_evaluate,
                                            y_strategy,
                                            l_reg,
                                            filter_n,
                                            kernel_size,
                                            pool_size,
                                            dense_n,
                                            batch_size,
                                            lr,
                                            optimizer,
                                            second_dense_n,
                                            data_augmentation,
                                            selective_data_aug,
                                            loss,
                                            block_list,
                                            global_avg
                                            ))
print("Number of training combinations that will now be evaluated:", len(train_combinations))

Number of training combinations that will now be evaluated: 1


In [ ]:
model = None
for model, strategy, y_strat, l_reg_value, n_filt, n_kern, n_pool, n_dense, n_batch, lr, optim, neuron_2nd_dense, data_aug, sel_data_aug, loss_type, n_blocks, g_avg in train_combinations:
    timestr = time.strftime("%Y%m%d-%H%M%S")

    config = {
        'batch_size': n_batch,
        'patience': 50,
        'epochs': 300,
        'lr': lr,
        'seed': my_seed,
        'l_reg': 0,
        'log_interval': 1,
        'model_name': model,
        'feature_norm': strategy,
        'y_strategy': y_strat,
        'dropout': True,
        'dataset': 'age',
        'embedding': 'mfcc_kaldi',
        'folder_fn': 'mfcc/age/',
        'mfcc_shape': (200, X[0].shape[1]),
        'data_augmentation': data_aug,
        'selective_data_aug': sel_data_aug,
        'kernel_initializer': 'glorot_normal',
        'loss': loss_type,
        'random_pick_mfcc': True,
        'generator on both train and test': True,
        'timestamp': timestr,
        'shuffle_temporal': None,
        'block_list': n_blocks,
        'lr_plateau': True,
        'lr_plateau_factor': 0.1,
        'lr_plateau_patience': 15,
        'relu_type': 'relu',
        'batch_norm': True,
        'global_average': g_avg,
        'reduce_mel': False,
        'n_categories': 8,
        'multi_output': True,
        'sampling_strategy': None,
        'without_initial_batch_norm': True,
        'cooldown': 5,
        'class_weights': None,
        'min_lr': 0.00001,
        'include_title_only_obs': True,
        'unbalanced': True,
        'unbalanced_include_title_only_obs': True

    }
    config['filter_n'] = n_filt
    config['kernel_size'] = n_kern
    config['pool_size'] = n_pool
    config['dense_n'] = n_dense
    config['optimizer'] = optim
    config['2nd_dense_n'] = neuron_2nd_dense
    config['strides'] = 1
    wandb.init(
        project='voxceleb_enrichment',
        name='_'.join([model, config['embedding'], strategy]),
        config=config
    )
    print(config)
    model = train_holdout(X, y, X_test, y_test, X_aug, y_aug, strategy, config['model_name'], config)


wandb: Currently logged in as: hechmik (use `wandb login --relogin` to force relogin)
/home/khaled/miniconda3/envs/tf/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.10.23 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'batch_size': 128, 'patience': 50, 'epochs': 300, 'lr': 0.01, 'seed': 19951008, 'l_reg': 0, 'log_interval': 1, 'model_name': 'cnn_resnet_1d', 'feature_norm': 'sub_mean_dataloader', 'y_strategy': '', 'dropout': True, 'dataset': 'age', 'embedding': 'mfcc_kaldi', 'folder_fn': 'mfcc/age/', 'mfcc_shape': (200, 30), 'data_augmentation': True, 'selective_data_aug': False, 'kernel_initializer': 'glorot_normal', 'loss': 'mse_plus_cross', 'random_pick_mfcc': True, 'generator on both train and test': True, 'timestamp': '20210328-002928', 'shuffle_temporal': None, 'block_list': [1, 1, 1], 'lr_plateau': True, 'lr_plateau_factor': 0.1, 'lr_plateau_patience': 15, 'relu_type': 'relu', 'batch_norm': True, 'global_average': True, 'reduce_mel': False, 'n_categories': 8, 'multi_output': True, 'sampling_strategy': None, 'without_initial_batch_norm': True, 'cooldown': 5, 'class_weights': None, 'min_lr': 1e-05, 'include_title_only_obs': True, 'unbalanced': True, 'unbalanced_include_title_only_obs': True, 'f

/home/khaled/miniconda3/envs/tf/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Train (92955,) Test (958, 200, 30) Resnet mode None
Epoch 1/300
727/727 [==============================] - 292s 384ms/step - loss: 86.1288 - regression_loss: 647.5984 - classifier_loss: 2.1369 - regression_mae: 19.3887 - classifier_accuracy: 0.2289 - val_loss: 38.9540 - val_regression_loss: 198.6782 - val_classifier_loss: 1.9086 - val_regression_mae: 11.9303 - val_classifier_accuracy: 0.2589
13940532
Epoch 2/300
727/727 [==============================] - 294s 404ms/step - loss: 35.9950 - regression_loss: 190.5483 - classifier_loss: 1.6940 - regression_mae: 11.2896 - classifier_accuracy: 0.3343 - val_loss: 35.9439 - val_regression_loss: 180.1736 - val_classifier_loss: 1.7927 - val_regression_mae: 11.0319 - val_classifier_accuracy: 0.2902
13966768
Epoch 3/300
727/727 [==============================] - 286s 393ms/step - loss: 34.1584 - regression_loss: 176.6162 - classifier_loss: 1.6497 - regression_mae: 10.7831 - classifier_accuracy: 0.3575 - val_loss: 38.7087 - val_regression_loss: 196.

In [ ]:
model.save("model-{}".format(config['timestamp']))
wandb.run.finish()

In [ ]:
wandb.run.finish()

## CNN 1D: Single input - single output

In [15]:
# Params
norm_strat_to_evaluate = ['sub_mean_dataloader']
y_strategy = ['']
l_reg = [0.0]
filter_n = [30]
kernel_size = [3]
pool_size = [(2)]
dense_n = [256]
batch_size = [128]
lr = [0.01]
optimizer = ['adam']
second_dense_n = [128]
data_augmentation = [True]
selective_data_aug = [False]
loss = ['mse']
block_list = [[1, 1, 1]]
global_avg = [True]
train_combinations = list(itertools.product(['cnn_resnet_1d'],
                                            norm_strat_to_evaluate,
                                            y_strategy,
                                            l_reg,
                                            filter_n,
                                            kernel_size,
                                            pool_size,
                                            dense_n,
                                            batch_size,
                                            lr,
                                            optimizer,
                                            second_dense_n,
                                            data_augmentation,
                                            selective_data_aug,
                                            loss,
                                            block_list,
                                            global_avg
                                            ))
print("Number of training combinations that will now be evaluated:", len(train_combinations))

Number of training combinations that will now be evaluated: 1


In [16]:
model = None
for model, strategy, y_strat, l_reg_value, n_filt, n_kern, n_pool, n_dense, n_batch, lr, optim, neuron_2nd_dense, data_aug, sel_data_aug, loss_type, n_blocks, g_avg in train_combinations:
    timestr = time.strftime("%Y%m%d-%H%M%S")

    config = {
        'batch_size': n_batch,
        'patience': 30,
        'epochs': 300,
        'lr': lr,
        'seed': my_seed,
        'l_reg': 0,
        'log_interval': 1,
        'model_name': model,
        'feature_norm': strategy,
        'y_strategy': y_strat,
        'dropout': True,
        'dataset': 'age',
        'embedding': 'mfcc_kaldi',
        'folder_fn': 'mfcc/age/',
        'mfcc_shape': (200, X[0].shape[1]),
        'data_augmentation': data_aug,
        'selective_data_aug': sel_data_aug,
        'kernel_initializer': 'glorot_normal',
        'loss': loss_type,
        'random_pick_mfcc': True,
        'generator on both train and test': True,
        'timestamp': timestr,
        'shuffle_temporal': None,
        'block_list': n_blocks,
        'lr_plateau': True,
        'lr_plateau_factor': 0.1,
        'lr_plateau_patience': 20,
        'relu_type': 'relu',
        'batch_norm': True,
        'global_average': g_avg,
        'reduce_mel': False,
        'n_categories': 0,
        'multi_output': None,
        'sampling_strategy': None,
        'without_initial_batch_norm': True,
        'cooldown': 10,
        'class_weights': None,
        'min_lr': 0.00001,
        'include_title_only_obs': True,
        'unbalanced': True,
        'unbalanced_include_title_only_obs': True

    }
    config['filter_n'] = n_filt
    config['kernel_size'] = n_kern
    config['pool_size'] = n_pool
    config['dense_n'] = n_dense
    config['optimizer'] = optim
    config['2nd_dense_n'] = neuron_2nd_dense
    config['strides'] = 1
    wandb.init(
        project='voxceleb_enrichment',
        name='_'.join([model, config['embedding'], strategy]),
        config=config
    )
    print(config)
    model = train_holdout(X, y, X_test, y_test, X_aug, y_aug, strategy, config['model_name'], config)


wandb: wandb version 0.10.23 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'batch_size': 128, 'patience': 30, 'epochs': 300, 'lr': 0.01, 'seed': 19951008, 'l_reg': 0, 'log_interval': 1, 'model_name': 'cnn_resnet_1d', 'feature_norm': 'sub_mean_dataloader', 'y_strategy': '', 'dropout': True, 'dataset': 'age', 'embedding': 'mfcc_kaldi', 'folder_fn': 'mfcc/age/', 'mfcc_shape': (200, 30), 'data_augmentation': True, 'selective_data_aug': False, 'kernel_initializer': 'glorot_normal', 'loss': 'mse', 'random_pick_mfcc': True, 'generator on both train and test': True, 'timestamp': '20210328-102247', 'shuffle_temporal': None, 'block_list': [1, 1, 1], 'lr_plateau': True, 'lr_plateau_factor': 0.1, 'lr_plateau_patience': 20, 'relu_type': 'relu', 'batch_norm': True, 'global_average': True, 'reduce_mel': False, 'n_categories': 0, 'multi_output': None, 'sampling_strategy': None, 'without_initial_batch_norm': True, 'cooldown': 10, 'class_weights': None, 'min_lr': 1e-05, 'include_title_only_obs': True, 'unbalanced': True, 'unbalanced_include_title_only_obs': True, 'filter_n': 

Train (92955,) Test (958, 200, 30) Resnet mode None
Epoch 1/300
727/727 [==============================] - 299s 393ms/step - loss: 595.8489 - mae: 19.0339 - mse: 595.8489 - val_loss: 181.8343 - val_mae: 11.1196 - val_mse: 181.8343
17141124
Epoch 2/300
727/727 [==============================] - 291s 400ms/step - loss: 191.1072 - mae: 11.3215 - mse: 191.1072 - val_loss: 181.2869 - val_mae: 11.1771 - val_mse: 181.2869
17151928
Epoch 3/300
727/727 [==============================] - 286s 394ms/step - loss: 176.7495 - mae: 10.8263 - mse: 176.7495 - val_loss: 248.6233 - val_mae: 13.1497 - val_mse: 248.6233
17158620
Epoch 4/300
727/727 [==============================] - 287s 394ms/step - loss: 169.8154 - mae: 10.5528 - mse: 169.8154 - val_loss: 316.4139 - val_mae: 14.7053 - val_mse: 316.4139
17158620
Epoch 5/300
727/727 [==============================] - 288s 395ms/step - loss: 162.6266 - mae: 10.2964 - mse: 162.6266 - val_loss: 179.2753 - val_mae: 10.7207 - val_mse: 179.2753
17158620
Epoch 6/

In [17]:
model.save("model-{}".format(config['timestamp']))
wandb.run.finish()

INFO:tensorflow:Assets written to: model-20210328-102247/assets


epoch,48
loss,103.74007
mae,8.0165
mse,103.74007
val_loss,171.47829
val_mae,10.3307
val_mse,171.47829
_step,48
_runtime,13765
_timestamp,1616929934
best_val_loss,162.22855


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▅▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
mse,█▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂▂▅█▂▄▁▃▂▁▃▁▁▂▁▁▃▃▁▃▂▂▃▁▂▂▂▂▁▂▁▂▂▂▁▁▂▂▂▁
val_mae,▃▃▆█▂▅▁▄▃▂▃▁▁▁▁▁▄▃▁▃▂▂▄▁▃▁▃▁▁▂▁▂▂▂▁▁▂▂▂▁
val_mse,▂▂▅█▂▄▁▃▂▁▃▁▁▂▁▁▃▃▁▃▂▂▃▁▂▂▂▂▁▂▁▂▂▂▁▁▂▂▂▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


## CNN 1D : decreasing filters

In [21]:
from importlib import reload
from src import resnets
from src import training_setup
reload(resnets)
reload(training_setup)

<module 'src.training_setup' from '/home/khaled/age_gender_recognition/asvtorch_modified/asvtorch/notebooks/src/training_setup.py'>

In [23]:
# Params
norm_strat_to_evaluate = ['sub_mean_dataloader']
y_strategy = ['']
l_reg = [0.0]
filter_n = [30]
kernel_size = [3]
pool_size = [(2)]
dense_n = [256]
batch_size = [128]
lr = [0.01]
optimizer = ['adam']
second_dense_n = [128]
data_augmentation = [True]
selective_data_aug = [False]
loss = ['mse']
block_list = [[1, 1, 1]]
global_avg = [True]
train_combinations = list(itertools.product(['cnn_resnet_1d'],
                                            norm_strat_to_evaluate,
                                            y_strategy,
                                            l_reg,
                                            filter_n,
                                            kernel_size,
                                            pool_size,
                                            dense_n,
                                            batch_size,
                                            lr,
                                            optimizer,
                                            second_dense_n,
                                            data_augmentation,
                                            selective_data_aug,
                                            loss,
                                            block_list,
                                            global_avg
                                            ))
print("Number of training combinations that will now be evaluated:", len(train_combinations))
model = None
for model, strategy, y_strat, l_reg_value, n_filt, n_kern, n_pool, n_dense, n_batch, lr, optim, neuron_2nd_dense, data_aug, sel_data_aug, loss_type, n_blocks, g_avg in train_combinations:
    timestr = time.strftime("%Y%m%d-%H%M%S")

    config = {
        'batch_size': n_batch,
        'patience': 30,
        'epochs': 300,
        'lr': lr,
        'seed': my_seed,
        'l_reg': 0,
        'log_interval': 1,
        'model_name': model,
        'feature_norm': strategy,
        'y_strategy': y_strat,
        'dropout': True,
        'dataset': 'age',
        'embedding': 'mfcc_kaldi',
        'folder_fn': 'mfcc/age/',
        'mfcc_shape': (200, X[0].shape[1]),
        'data_augmentation': data_aug,
        'selective_data_aug': sel_data_aug,
        'kernel_initializer': 'glorot_normal',
        'loss': loss_type,
        'random_pick_mfcc': True,
        'generator on both train and test': True,
        'timestamp': timestr,
        'shuffle_temporal': None,
        'block_list': n_blocks,
        'lr_plateau': True,
        'lr_plateau_factor': 0.1,
        'lr_plateau_patience': 20,
        'relu_type': 'relu',
        'batch_norm': True,
        'global_average': g_avg,
        'reduce_mel': False,
        'n_categories': 0,
        'multi_output': None,
        'sampling_strategy': None,
        'without_initial_batch_norm': True,
        'cooldown': 10,
        'class_weights': None,
        'min_lr': 0.00001,
        'include_title_only_obs': True,
        'unbalanced': True,
        'unbalanced_include_title_only_obs': True,
        'decreasing_filters': True

    }
    config['filter_n'] = n_filt
    config['kernel_size'] = n_kern
    config['pool_size'] = n_pool
    config['dense_n'] = n_dense
    config['optimizer'] = optim
    config['2nd_dense_n'] = neuron_2nd_dense
    config['strides'] = 1
    wandb.init(
        project='voxceleb_enrichment',
        name='_'.join([model, config['embedding'], strategy]),
        config=config
    )
    print(config)
    model = train_holdout(X, y, X_test, y_test, X_aug, y_aug, strategy, config['model_name'], config)


Number of training combinations that will now be evaluated: 1


wandb: wandb version 0.10.23 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'batch_size': 128, 'patience': 30, 'epochs': 300, 'lr': 0.01, 'seed': 19951008, 'l_reg': 0, 'log_interval': 1, 'model_name': 'cnn_resnet_1d', 'feature_norm': 'sub_mean_dataloader', 'y_strategy': '', 'dropout': True, 'dataset': 'age', 'embedding': 'mfcc_kaldi', 'folder_fn': 'mfcc/age/', 'mfcc_shape': (200, 30), 'data_augmentation': True, 'selective_data_aug': False, 'kernel_initializer': 'glorot_normal', 'loss': 'mse', 'random_pick_mfcc': True, 'generator on both train and test': True, 'timestamp': '20210328-145849', 'shuffle_temporal': None, 'block_list': [1, 1, 1], 'lr_plateau': True, 'lr_plateau_factor': 0.1, 'lr_plateau_patience': 20, 'relu_type': 'relu', 'batch_norm': True, 'global_average': True, 'reduce_mel': False, 'n_categories': 0, 'multi_output': None, 'sampling_strategy': None, 'without_initial_batch_norm': True, 'cooldown': 10, 'class_weights': None, 'min_lr': 1e-05, 'include_title_only_obs': True, 'unbalanced': True, 'unbalanced_include_title_only_obs': True, 'decreasing_

Train (92955,) Test (958, 200, 30) Resnet mode None
Epoch 1/300
727/727 [==============================] - 195s 249ms/step - loss: 610.5267 - mae: 19.0314 - mse: 610.5267 - val_loss: 207.1350 - val_mae: 11.5085 - val_mse: 207.1350
20544912
Epoch 2/300
727/727 [==============================] - 174s 239ms/step - loss: 195.6135 - mae: 11.4744 - mse: 195.6135 - val_loss: 215.9154 - val_mae: 12.4918 - val_mse: 215.9154
20545184
Epoch 3/300
727/727 [==============================] - 181s 249ms/step - loss: 177.9529 - mae: 10.8646 - mse: 177.9529 - val_loss: 171.6050 - val_mae: 10.9044 - val_mse: 171.6050
20545184
Epoch 4/300
727/727 [==============================] - 185s 255ms/step - loss: 174.7715 - mae: 10.7415 - mse: 174.7715 - val_loss: 219.6935 - val_mae: 12.6061 - val_mse: 219.6935
20545184
Epoch 5/300
727/727 [==============================] - 178s 245ms/step - loss: 168.6065 - mae: 10.5168 - mse: 168.6065 - val_loss: 216.1697 - val_mae: 12.3127 - val_mse: 216.1697
20545184
Epoch 6/

In [24]:
model.save("model-{}".format(config['timestamp']))
wandb.run.finish()

INFO:tensorflow:Assets written to: model-20210328-145849/assets


epoch,57
loss,118.54803
mae,8.68172
mse,118.54803
val_loss,165.0312
val_mae,10.44685
val_mse,165.0312
_step,57
_runtime,10482
_timestamp,1616943213
best_val_loss,161.38457


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▄▅▂▅▅▃▄▅▂▂▂▁▂▂▂▂▃▂▁▁▃▃▃▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁
val_mae,▄▆▃▆▅▄▆▆▃▂▂▂▃▄▃▂▄▂▂▁▄▄▄▁▂▁▂▂█▂▂▂▁▂▂▂▂▂▂▂
val_mse,▄▅▂▅▅▃▄▅▂▂▂▁▂▂▂▂▃▂▁▁▃▃▃▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


## CNN 1-D: Multi in - multi-out decreasing

In [25]:
# Params
norm_strat_to_evaluate = ['sub_mean_dataloader']
y_strategy = ['']
l_reg = [0.0]
filter_n = [30]
kernel_size = [3]
pool_size = [(2)]
dense_n = [256]
batch_size = [128]
lr = [0.01]
optimizer = ['adam']
second_dense_n = [128]
data_augmentation = [True]
selective_data_aug = [False]
loss = ['mse_plus_cross']
block_list = [[1, 1, 1]]
global_avg = [True]
train_combinations = list(itertools.product(['cnn_resnet_1d'],
                                            norm_strat_to_evaluate,
                                            y_strategy,
                                            l_reg,
                                            filter_n,
                                            kernel_size,
                                            pool_size,
                                            dense_n,
                                            batch_size,
                                            lr,
                                            optimizer,
                                            second_dense_n,
                                            data_augmentation,
                                            selective_data_aug,
                                            loss,
                                            block_list,
                                            global_avg
                                            ))
print("Number of training combinations that will now be evaluated:", len(train_combinations))
model = None
for model, strategy, y_strat, l_reg_value, n_filt, n_kern, n_pool, n_dense, n_batch, lr, optim, neuron_2nd_dense, data_aug, sel_data_aug, loss_type, n_blocks, g_avg in train_combinations:
    timestr = time.strftime("%Y%m%d-%H%M%S")

    config = {
        'batch_size': n_batch,
        'patience': 30,
        'epochs': 300,
        'lr': lr,
        'seed': my_seed,
        'l_reg': 0,
        'log_interval': 1,
        'model_name': model,
        'feature_norm': strategy,
        'y_strategy': y_strat,
        'dropout': True,
        'dataset': 'age',
        'embedding': 'mfcc_kaldi',
        'folder_fn': 'mfcc/age/',
        'mfcc_shape': (200, X[0].shape[1]),
        'data_augmentation': data_aug,
        'selective_data_aug': sel_data_aug,
        'kernel_initializer': 'glorot_normal',
        'loss': loss_type,
        'random_pick_mfcc': True,
        'generator on both train and test': True,
        'timestamp': timestr,
        'shuffle_temporal': None,
        'block_list': n_blocks,
        'lr_plateau': True,
        'lr_plateau_factor': 0.1,
        'lr_plateau_patience': 20,
        'relu_type': 'relu',
        'batch_norm': True,
        'global_average': g_avg,
        'reduce_mel': False,
        'n_categories': 8,
        'multi_output': True,
        'sampling_strategy': None,
        'without_initial_batch_norm': True,
        'cooldown': 10,
        'class_weights': None,
        'min_lr': 0.00001,
        'include_title_only_obs': True,
        'unbalanced': True,
        'unbalanced_include_title_only_obs': True,
        'decreasing_filters': True

    }
    config['filter_n'] = n_filt
    config['kernel_size'] = n_kern
    config['pool_size'] = n_pool
    config['dense_n'] = n_dense
    config['optimizer'] = optim
    config['2nd_dense_n'] = neuron_2nd_dense
    config['strides'] = 1
    wandb.init(
        project='voxceleb_enrichment',
        name='_'.join([model, config['embedding'], strategy]),
        config=config
    )
    print(config)
    model = train_holdout(X, y, X_test, y_test, X_aug, y_aug, strategy, config['model_name'], config)


Number of training combinations that will now be evaluated: 1


wandb: wandb version 0.10.23 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'batch_size': 128, 'patience': 30, 'epochs': 300, 'lr': 0.01, 'seed': 19951008, 'l_reg': 0, 'log_interval': 1, 'model_name': 'cnn_resnet_1d', 'feature_norm': 'sub_mean_dataloader', 'y_strategy': '', 'dropout': True, 'dataset': 'age', 'embedding': 'mfcc_kaldi', 'folder_fn': 'mfcc/age/', 'mfcc_shape': (200, 30), 'data_augmentation': True, 'selective_data_aug': False, 'kernel_initializer': 'glorot_normal', 'loss': 'mse_plus_cross', 'random_pick_mfcc': True, 'generator on both train and test': True, 'timestamp': '20210328-185850', 'shuffle_temporal': None, 'block_list': [1, 1, 1], 'lr_plateau': True, 'lr_plateau_factor': 0.1, 'lr_plateau_patience': 20, 'relu_type': 'relu', 'batch_norm': True, 'global_average': True, 'reduce_mel': False, 'n_categories': 8, 'multi_output': True, 'sampling_strategy': None, 'without_initial_batch_norm': True, 'cooldown': 10, 'class_weights': None, 'min_lr': 1e-05, 'include_title_only_obs': True, 'unbalanced': True, 'unbalanced_include_title_only_obs': True, '

/home/khaled/miniconda3/envs/tf/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Train label encoder output: (92955,)
Test label encoder output: (958,)
Train (92955,) Test (958, 200, 30) Resnet mode None
Epoch 1/300
727/727 [==============================] - 186s 237ms/step - loss: 77.2242 - regression_loss: 556.4829 - classifier_loss: 2.1576 - regression_mae: 18.4482 - classifier_accuracy: 0.2114 - val_loss: 38.0260 - val_regression_loss: 189.3863 - val_classifier_loss: 1.9087 - val_regression_mae: 11.1988 - val_classifier_accuracy: 0.2390
23922136
Epoch 2/300
727/727 [==============================] - 166s 229ms/step - loss: 36.0732 - regression_loss: 190.7292 - classifier_loss: 1.7000 - regression_mae: 11.2883 - classifier_accuracy: 0.3372 - val_loss: 35.4166 - val_regression_loss: 178.5920 - val_classifier_loss: 1.7557 - val_regression_mae: 11.1118 - val_classifier_accuracy: 0.2891 regression_loss: 192.7761 - c - ETA: 1:03 - loss: 36.3323 - regression_loss: 192.4515 - classifier_loss: 1.7087 - regression_mae: 11.3430 - classifier_accuracy: 0.33 - ETA: 1:03 - lo

In [26]:
model.save("model-{}".format(config['timestamp']))
wandb.run.finish()

INFO:tensorflow:Assets written to: model-20210328-185850/assets


epoch,41
loss,26.98375
regression_loss,121.17906
classifier_loss,1.48658
regression_mae,8.76789
classifier_accuracy,0.42725
val_loss,34.11488
val_regression_loss,168.01892
val_classifier_loss,1.7313
val_regression_mae,10.50871
val_classifier_accuracy,0.3476


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,█▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
regression_loss,█▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
classifier_loss,█▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
regression_mae,█▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
classifier_accuracy,▁▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████
val_loss,▄▃▂▃▂▄▂▂▂▁▃▁▂▂▃▃█▄▁▂▃▆▄▂▂▁▁█▁▂▃▁▁▂▁▂▁▁▂▂
val_regression_loss,▃▃▂▃▂▄▃▂▂▁▃▁▂▂▃▃▇▄▂▂▃▅▄▂▂▁▁█▁▂▂▁▁▁▁▂▁▁▂▂
val_classifier_loss,▆▃▂▃▂▃▁▃▂▁▂▁▃▃▄▄█▆▁▃▄▇▅▁▁▁▁▇▁▂▃▂▂▂▁▂▂▂▂▂
val_regression_mae,▄▄▂▄▂▃▂▂▂▂▃▁▂▂▄▄▇▅▁▃▄▆▅▂▁▁▁█▁▂▃▂▂▂▁▂▂▂▂▂
val_classifier_accuracy,▁▄▆▃▇▆▇▆▆▇▆▇▅▆▄▆▃▂▇▆▆▄▄▇▇██▄▇▇▆▇█▇█▇▇▇█▇


# NO Data augm:

# Evaluate models
## CNN 1-D : Single input - single output

In [3]:
from tensorflow import keras
model = keras.models.load_model('model-20210328-102247/', compile='false')

In [4]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 200, 30)]    0                                            
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 200, 30)      2730        input_2[0][0]                    
__________________________________________________________________________________________________
re_lu_9 (ReLU)                  (None, 200, 30)      0           conv1d_7[0][0]                   
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 200, 30)      120         re_lu_9[0][0]                    
____________________________________________________________________________________________

In [5]:
import numpy as np

In [9]:
%%time
data = np.load('/media/hdd1/khaled/npz_files/final_version/test_data.npz', allow_pickle=True)

vectors = []
for x in list(data.keys()):
    vectors.append(data[x])
X_test, y_test, X_spk_video_labels_test = vectors
data.close()
data = None
vectors = None
del data, vectors


CPU times: user 1.83 s, sys: 359 ms, total: 2.19 s
Wall time: 2.19 s


In [12]:
X_test_mel = empty(X_test.shape, dtype='object')
for i in range(X_test.shape[0]):
    X_test_mel[i] = scipy.fftpack.idct(X_test[i])
X_spk_labels_test = [''.join(x.split('-')[1:]) for x in X_spk_video_labels_test]
test_ids_balanced = get_correct_recordings_index(X_spk_labels_test)
len(test_ids_balanced)
X_test = X_test[test_ids_balanced]
y_test = y_test[test_ids_balanced]
X_test_mel = X_test_mel[test_ids_balanced]

get_correct_recordings_index >>>
1
get_correct_recordings_index <<<


In [13]:
y_pred_avg = []
for i, test_track in enumerate(tqdm(X_test)):
    n_slice_to_compute = test_track.shape[0] - 200
    current_track_pred = []
    for idx in range(0, n_slice_to_compute, 100):        
        sliced_track = test_track[idx:idx+200,:]
        #print(sliced_track.shape)
        sliced_track = sliced_track - np.mean(sliced_track,axis=0)
        #print(sliced_track.shape)
        slice_pred = model.predict(sliced_track.reshape(1, 200, 30))
        current_track_pred.append(slice_pred)
    # Last prediction:
    sliced_track = test_track[-200:,:]
    sliced_track = sliced_track - np.mean(sliced_track,axis=0)
    #print(sliced_track.shape)
    slice_pred = model.predict(sliced_track.reshape(1, 200, 30))
    current_track_pred.append(slice_pred)
    y_pred_avg.append(np.mean(current_track_pred))

100%|██████████| 958/958 [08:03<00:00,  1.98it/s]


In [14]:
import json
np.savetxt("y_pred-cnn_single_single.txt", y_pred_avg)

In [15]:
np.mean(np.abs(np.array(y_pred_avg) - y_test))

9.442898750305176

In [79]:
n_steps = []
for i, test_track in enumerate(tqdm(X_test)):
    n_slice_to_compute = test_track.shape[0] - 200
    n_steps.append(n_slice_to_compute)

100%|██████████| 958/958 [00:00<00:00, 211871.51it/s]


In [80]:
np.sum(n_steps)

482316

In [81]:
np.mean(n_steps)

503.4613778705637

In [82]:
np.std(n_steps)

517.7237759810298

## CNN 1D - Multiout

In [16]:
from tensorflow import keras
model = keras.models.load_model('model-20210328-002928/', compile='false')

In [17]:
y_pred_avg = []
for i, test_track in enumerate(tqdm(X_test)):
    n_slice_to_compute = test_track.shape[0] - 200
    current_track_pred = []
    for idx in range(0, n_slice_to_compute, 100):        
        sliced_track = test_track[idx:idx+200,:]
        #print(sliced_track.shape)
        sliced_track = sliced_track - np.mean(sliced_track,axis=0)
        #print(sliced_track.shape)
        slice_pred = model.predict(sliced_track.reshape(1, 200, 30))[0]
        current_track_pred.append(slice_pred)
    # Last prediction:
    sliced_track = test_track[-200:,:]
    sliced_track = sliced_track - np.mean(sliced_track,axis=0)
    #print(sliced_track.shape)
    slice_pred = model.predict(sliced_track.reshape(1, 200, 30))[0]
    current_track_pred.append(slice_pred)
    y_pred_avg.append(np.mean(current_track_pred))

100%|██████████| 958/958 [08:08<00:00,  1.96it/s]


In [18]:
np.mean(np.abs(np.array(y_pred_avg) - y_test))

9.510689193868936

In [19]:
1

1

## Power mel

In [20]:
%%time
model = keras.models.load_model('model-20210328-223050/', compile='false')
y_pred_avg = []
for i, test_track in enumerate(tqdm(X_test)):
    n_slice_to_compute = test_track.shape[0] - 200
    current_track_pred = []
    for idx in range(0, n_slice_to_compute, 100):        
        sliced_track = test_track[idx:idx+200,:]
        #print(sliced_track.shape)
        sliced_track = sliced_track - np.mean(sliced_track,axis=0)
        #print(sliced_track.shape)
        slice_pred = model.predict(sliced_track.reshape(1, 200, 30))[0]
        current_track_pred.append(slice_pred)
    # Last prediction:
    sliced_track = test_track[-200:,:]
    sliced_track = sliced_track - np.mean(sliced_track,axis=0)
    #print(sliced_track.shape)
    slice_pred = model.predict(sliced_track.reshape(1, 200, 30))[0]
    current_track_pred.append(slice_pred)
    y_pred_avg.append(np.mean(current_track_pred))

100%|██████████| 958/958 [08:48<00:00,  1.81it/s]

CPU times: user 8min 25s, sys: 10.5 s, total: 8min 35s
Wall time: 8min 54s


In [21]:
np.mean(np.abs(np.array(y_pred_avg) - y_test))

13.325782783842783